In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


df = pd.read_csv('https://raw.githubusercontent.com/nirvannar/UdacityProject/main/GVP_Volcano_List_Holocene1.csv',encoding='latin-1')


In [ ]:
df.head() # Explore the first few rows of the dataset

In [ ]:
df.describe() # Summary statistics associated with the quantitative variables


In [ ]:
df.isnull().any() # Check for columns with blanks by calculating the number of null values

In [ ]:
import datapane as dp #Interactive data table in Medium

df = pd.read_csv('https://raw.githubusercontent.com/nirvannar/UdacityProject/main/GVP_Volcano_List_Holocene1.csv',encoding='latin-1')

table = dp.DataTable(df)
report = dp.Report(table)
report.upload(name='Volcano data')

In [ ]:
df.hist();

In [ ]:
sns.pairplot(df)

In [ ]:
plt.figure(figsize=(12,6))
sns.heatmap(df.corr(), annot=True)

In [ ]:
print(df[df['Elevation (m)']==df['Elevation (m)'].max()]) # Highest Volcano

In [ ]:
df[df['Elevation (m)']==df['Elevation (m)'].max()] # Highest Volcano

In [ ]:
df['Country'].value_counts(normalize=True) # Most volcanoes per country

In [ ]:
df['Primary Volcano Type'].value_counts(normalize=True) # Volcano Types in descending order

In [ ]:
df['Region'].value_counts(normalize=True) # Most volcanoes per region

In [ ]:
df['Last Known Eruption'].value_counts() # Eruptions per year observed in descending order


In [ ]:
pip install wordcloud

In [ ]:
from wordcloud import WordCloud

text= ' '
for s, row in df.iterrows():
    text = " ".join([text,"".join(row['Country'].strip().split(" "))])
    
text = text.strip()



plt.figure(figsize=(12,6))
wordcloud = WordCloud(width=600, height=300, max_font_size=70, max_words=20, collocations=False).generate(text)
wordcloud.recolor(random_state=0)
plt.imshow(wordcloud, interpolation='bilinear')
plt.title("Top 20 Countries with most volcanic eruptions", fontsize=30)
plt.axis("off")
plt.show()
wordcloud.to_file('wordcloud.png')

image = dp.File(file='wordcloud.png')
dp.Report(image).upload(name='Country',
                        description='Top 20 Countries with most volcanic eruptions')

In [ ]:
targets = df.loc[df['Last Known Eruption'] == "2021 CE", ['Volcano Name']] #Volcanoes active this year
print(targets)


In [ ]:
# Closest Volcano to Johannesburg

from math import radians, cos, sin, asin, sqrt
def dist(Longitude1,Latitude1,Longitude2,Latitude2 ):
   
    # convert decimal degrees to radians 
    Longitude1,Latitude1, Longitude2,Latitude2  = map(radians, [Longitude1, Latitude1,Longitude2,Latitude2])
    # haversine formula 
    dlon = Longitude2 - Longitude1  
    dlat = Latitude2 - Latitude1 
    a = sin(dlat/2)**2 + cos(Latitude1) * cos(Latitude2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

df['distance'] = [dist(df.Longitude[i],df.Latitude[i], 28.2411459,-26.1366728) 
                  for i in range(len(df))]
df['distance'] = df['distance'].round(decimals=3)

df.head(296)

print(df[df['distance']==df['distance'].min()])


In [ ]:
df.head() # Check if column for distance is appended

In [ ]:
df.loc[(df['Last Known Eruption'] == "2021 CE") & (df['distance'] <3000)] #Active volcanoes within 3000km radius of Jhb

In [ ]:
pip install -U datapane

In [ ]:
# add marker one by one on the map to indicate location of Holocene Volcanoes
import pandas as pd
import folium

import datapane as dp

dp.login(token='627d7678b242daf7ecba23b579eab5e6754290b8')


df = pd.read_csv('https://raw.githubusercontent.com/nirvannar/UdacityProject/main/GVP_Volcano_List_Holocene1.csv',encoding='latin-1')

m = folium.Map(location=[-26.1366728, 28.2411459], tiles="OpenStreetMap", zoom_start=3)


#for i in range(0,len(data)):
for index, row in df.iterrows():
    icon_path = r"C:/Users/nrampersad/Downloads/noun_Volcano_1708.png"
    icon = folium.features.CustomIcon(icon_image=icon_path ,icon_size=(20,20))
    folium.Marker(
              #location=[data.iloc[i]['lat'], data.iloc[i]['lon']],
              #popup=data.iloc[i]['name'],
              [row['Latitude'], row['Longitude']], 
                  popup=row['Volcano Name'],
              icon=icon
              ).add_to(m)

# Show the map again
dp.Report(dp.Plot(m)).upload(name='folium_map')
m